In [1]:
import torch
import cv2
import numpy as np
from torchreid.reid.utils import FeatureExtractor
import pickle

/Users/ooj/Dev/HKUST/fyp/FinalYearProject/src/cv/.venv/lib/python3.11/site-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [3]:
extractor = FeatureExtractor(
    model_name='osnet_x1_0',
    model_path=None,
    device=device
)

Successfully loaded imagenet pretrained weights from "/Users/ooj/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352


In [4]:
img = cv2.imread("media/bus.jpg")

In [5]:
features = extractor([img])
print("Extracted embedding shape:", features.shape)
print("Embedding vector (first 10 dims):", features[0][:10])

Extracted embedding shape: torch.Size([1, 512])
Embedding vector (first 10 dims): tensor([0.0000, 2.1946, 1.4717, 0.0000, 2.2121, 1.3011, 2.6642, 1.5508, 0.0000,
        1.7066], device='mps:0')


In [6]:
cap = cv2.VideoCapture(0)
embeddings = []
count = 0

In [7]:
while count < 5:
    ret, frame = cap.read()
    if not ret:
        continue
    
    cv2.imshow("Enrollment - Look at the camera", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('c'):
        embedding_tensor = extractor([frame])
        embedding = embedding_tensor.cpu().numpy().flatten()
        norm = np.linalg.norm(embedding)
        embedding = embedding if norm == 0 else embedding / norm
        embeddings.append(embedding)
        count += 1
        print(f"Captured sample {count}/{5}")
    
    elif key == ord('q'):
        break


Captured sample 1/5
Captured sample 2/5
Captured sample 3/5
Captured sample 4/5
Captured sample 5/5


In [8]:
cap.release()
cv2.destroyAllWindows()

In [9]:
filename = f"test2_embeddings.pkl"
with open(filename, "wb") as f:
    pickle.dump(embeddings, f)
print(f"Enrollment complete. Embeddings saved to {filename}")

Enrollment complete. Embeddings saved to test2_embeddings.pkl
